In [56]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.checkpoint.sqlite import SqliteSaver

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

config = {
    "configurable": {
        "thread_id": "0_x",
    },
}

In [57]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [58]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response],
    }

In [59]:
graph_builder.add_node("chatbot", chatbot)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [60]:
result = graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I live in Europe now. And the city I live in is Gothenburg.",
            },
        ]
    },
    config=config,
)

In [61]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I live in Europe now. And the city I live in is Gothenburg.
================================== Ai Message ==================================

That's great! Gothenburg is a beautiful city in Sweden known for its picturesque canals, vibrant cultural scene, and unique blend of historical and modern architecture. Are there specific aspects of Gothenburg you are interested in, such as places to visit, activities, or local food?
================================ Human Message =================================

I live in Europe now. And the city I live in is Zagreb.
================================ Human Message =================================

I live in Europe now. And the city I live in is Gothenburg.
================================== Ai Message ==================================

Gothenburg is a wonderful city with a lot to offer! It has a rich maritime history and is known for its beautiful parks, the famo

In [62]:
state_history = graph.get_state_history(config)

In [63]:
for state_snapshot in list(state_history):
    print(state_snapshot.next)
    print(state_snapshot.values["messages"])
    print("=========\n")

()
[HumanMessage(content='I live in Europe now. And the city I live in is Gothenburg.', additional_kwargs={}, response_metadata={}, id='31a187fd-e89d-456a-8576-3fc4e5d31e73'), AIMessage(content="That's great! Gothenburg is a beautiful city in Sweden known for its picturesque canals, vibrant cultural scene, and unique blend of historical and modern architecture. Are there specific aspects of Gothenburg you are interested in, such as places to visit, activities, or local food?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 23, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DCW7zA0NofFVvfsyo38XtrkTqvLsL', 'service_tier': 'default', 'finish_reason': 'st

In [64]:
state_history = graph.get_state_history(config)

to_fork = list(state_history)[-5]

to_fork.config

{'configurable': {'thread_id': '0_x',
  'checkpoint_ns': '',
  'checkpoint_id': '1f110f03-cbb2-6d50-8002-592c35837483'}}

In [66]:
from langchain_core.messages import HumanMessage

graph.update_state(
    to_fork.config,
    {
        "messages": [
            HumanMessage(
                content="I live in Europe now. And the city I live in is Barcelona.",
                id="'1f110f03-cbb2-6d50-8002-592c35837483",
            )
        ]
    },
)

{'configurable': {'thread_id': '0_x',
  'checkpoint_ns': '',
  'checkpoint_id': '1f110f09-59b1-62f0-8003-b45ed9c02f03'}}

In [67]:
from langchain_core.messages import HumanMessage

graph.update_state(
    to_fork.config,
    {
        "messages": [
            HumanMessage(
                content="I live in Europe now. And the city I live in is Zagreb.",
                id="1f110f09-59b1-62f0-8003-b45ed9c02f03",
            )
        ]
    },
)

{'configurable': {'thread_id': '0_x',
  'checkpoint_ns': '',
  'checkpoint_id': '1f110f09-a840-62c2-8003-d16e305f30f8'}}

In [68]:
forked_state = graph.get_state_history(
    {
        "configurable": {
            "thread_id": "11",
            "checkpoint_ns": "",
            "checkpoint_id": "1f110f09-a840-62c2-8003-d16e305f30f8",
        }
    }
)

list(forked_state)


[]

In [69]:
result = graph.invoke(
    None,
    {

        "configurable": {
            "thread_id": "0_x",
            "checkpoint_ns": "",
            "checkpoint_id": "1f110f09-59b1-62f0-8003-b45ed9c02f03",
        }
        
    },
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I live in Europe now. And the city I live in is Gothenburg.
================================== Ai Message ==================================

That's great! Gothenburg is a beautiful city in Sweden known for its picturesque canals, vibrant cultural scene, and unique blend of historical and modern architecture. Are there specific aspects of Gothenburg you are interested in, such as places to visit, activities, or local food?
================================ Human Message =================================

I live in Europe now. And the city I live in is Zagreb.
================================ Human Message =================================

I live in Europe now. And the city I live in is Barcelona.
